In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Final") \
    .getOrCreate()

sc = spark.sparkContext

24/12/24 13:16:57 WARN Utils: Your hostname, mac.local resolves to a loopback address: 127.0.0.1; using 10.0.0.252 instead (on interface en0)
24/12/24 13:16:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 13:16:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### 1. Data Ingestion
#### 2. Feature Selection
#### 3. Data Checks
#####     - check data type
#####     - check missing value
#####     - check unique values of each column
#####     - check statistics of dataset
#### 4. Data Saving/Loading
#### 5. Train Test Split
#### 6. Feature Engineering/Transformation
#### 7. Model Training and evaluation

### 1. Data Extraction

In [3]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [4]:
df = spark.read.text('2020_lar.txt')

In [5]:
df = spark.read.option("header", "true") \
    .option("delimiter", "|") \
    .csv('2020_lar.txt')

In [6]:
df

24/12/24 13:17:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+--------------------+--------------+----------+-----------+------------+---------------------+-------------------------+-------------------------+--------------------+------------------+-----------------+------------+--------------+-----------+---------+------------+-----------+----------------+-----------------------+------------------------------+-----------+----------------------------+-------------+-----------+------------+----------------+---------------------+-------------------+---------------+--------------+---------+-----------------------+-----------------+---------------------+---------------------+---------------+----------------------------+--------------+-------------------+--------------+---------------------------------------+----------------------------------------+-----------+----------------------------+------+--------------------+---------------------------+------------------------------+---------------------+---------------------+---------------------+---------------------+---------------------+------------------------+------------------------+------------------------+------------------------+------------------------+----------------------------+-------------------------------+----------------+----------------+----------------+----------------+----------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+--------------------------+-------------+----------------+----------------------+-------------------------+-------------+----------------+----------------------+-------------------------+-------------------------+--------------------------------+-----+-----+-----+-----+-----+---------------+---------------+---------------+---------------+----------------+---------------------------------+---------------------------------+------------------------------+--------------------------+------------------------------+---------------------------------+
|activity_year|                 lei|derived_msa_md|state_code|county_code|census_tract|conforming_loan_limit|derived_loan_product_type|derived_dwelling_category|   derived_ethnicity|      derived_race|      derived_sex|action_taken|purchaser_type|preapproval|loan_type|loan_purpose|lien_status|reverse_mortgage|open_end_line_of_credit|business_or_commercial_purpose|loan_amount|combined_loan_to_value_ratio|interest_rate|rate_spread|hoepa_status|total_loan_costs|total_points_and_fees|origination_charges|discount_points|lender_credits|loan_term|prepayment_penalty_term|intro_rate_period|negative_amortization|interest_only_payment|balloon_payment|other_nonamortizing_features|property_value|construction_method|occupancy_type|manufactured_home_secured_property_type|manufactured_home_land_property_interest|total_units|multifamily_affordable_units|income|debt_to_income_ratio|applicant_credit_score_type|co_applicant_credit_score_type|applicant_ethnicity_1|applicant_ethnicity_2|applicant_ethnicity_3|applicant_ethnicity_4|applicant_ethnicity_5|co_applicant_ethnicity_1|co_applicant_ethnicity_2|co_applicant_ethnicity_3|co_applicant_ethnicity_4|co_applicant_ethnicity_5|applicant_ethnicity_observed|co_applicant_ethnicity_observed|applicant_race_1|applicant_race_2|applicant_race_3|applicant_race_4|applicant_race_5|co_applicant_race_1|co_applicant_race_2|co_applicant_race_3|co_applicant_race_4|co_applicant_race_5|applicant_race_observed|co_applicant_race_observed|applicant_sex|co_applicant_sex|applicant_sex_observed|co_applicant_sex_observed|applicant_age|co_applicant_age|applicant_age_above_62|co_applicant_age_above_62|submission_of_application|initially_payable_to_institution|aus_1|aus_2|aus_3|aus_4|aus_5|denial_reason_1|denial_reason_2|denial_reason_3|denial_reason_4|tract_population|tract_minority_population_percent|ffiec_msa_md_median_family_income|tract_to_msa_income_percentage|tract_owner_occupied_units|tract_one_to_four_family_homes|tract_median_age_of_housing_units|
+-------------+-----

In [7]:
total_rows = df.count()

In [8]:
total_rows

25699043

In [9]:
df.columns

['activity_year',
 'lei',
 'derived_msa_md',
 'state_code',
 'county_code',
 'census_tract',
 'conforming_loan_limit',
 'derived_loan_product_type',
 'derived_dwelling_category',
 'derived_ethnicity',
 'derived_race',
 'derived_sex',
 'action_taken',
 'purchaser_type',
 'preapproval',
 'loan_type',
 'loan_purpose',
 'lien_status',
 'reverse_mortgage',
 'open_end_line_of_credit',
 'business_or_commercial_purpose',
 'loan_amount',
 'combined_loan_to_value_ratio',
 'interest_rate',
 'rate_spread',
 'hoepa_status',
 'total_loan_costs',
 'total_points_and_fees',
 'origination_charges',
 'discount_points',
 'lender_credits',
 'loan_term',
 'prepayment_penalty_term',
 'intro_rate_period',
 'negative_amortization',
 'interest_only_payment',
 'balloon_payment',
 'other_nonamortizing_features',
 'property_value',
 'construction_method',
 'occupancy_type',
 'manufactured_home_secured_property_type',
 'manufactured_home_land_property_interest',
 'total_units',
 'multifamily_affordable_units',
 'in

In [10]:
# sample subset for local machine
fraction = 10000/total_rows
df = df.sample(fraction=fraction, withReplacement=False)

### 2. Feature Selection

In [10]:
loan_details_df = df.select("activity_year", "lei", "loan_type","loan_purpose", "loan_amount", "interest_rate","loan_term", "action_taken")
borrower_details_df = df.select("income","applicant_age","applicant_sex", "applicant_credit_score_type", "co_applicant_age", "co_applicant_credit_score_type")
property_details_df = df.select("derived_msa_md", "state_code", "county_code", "property_value", "total_units", "occupancy_type")


In [11]:
columns = loan_details_df.columns + borrower_details_df.columns + property_details_df.columns

In [12]:
len(columns)

20

In [13]:
df = df.select(columns)

In [14]:
df

+-------------+--------------------+---------+------------+-----------+-------------+---------+------------+------+-------------+-------------+---------------------------+----------------+------------------------------+--------------+----------+-----------+--------------+-----------+--------------+
|activity_year|                 lei|loan_type|loan_purpose|loan_amount|interest_rate|loan_term|action_taken|income|applicant_age|applicant_sex|applicant_credit_score_type|co_applicant_age|co_applicant_credit_score_type|derived_msa_md|state_code|county_code|property_value|total_units|occupancy_type|
+-------------+--------------------+---------+------------+-----------+-------------+---------+------------+------+-------------+-------------+---------------------------+----------------+------------------------------+--------------+----------+-----------+--------------+-----------+--------------+
|         2020|549300RBX56T2MW5HO19|        1|           1|     135000|        3.875|      360|           1|    65|        25-34|            1|                          2|           25-34|                             5|         29620|        MI|      26065|        145000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|          31|     315000|        2.124|      180|           1|   135|        45-54|            1|                          5|           45-54|                             1|         31084|        CA|      06037|        855000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        2|           1|     135000|         3.43|       NA|           1|    NA|        65-74|            1|                          9|           65-74|                             9|         41100|        UT|      49053|        255000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|          32|     405000|           NA|      240|           4|   120|        35-44|            1|                          9|           35-44|                             9|         19740|        CO|      08031|            NA|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|           1|     435000|          2.5|      360|           1|   328|        55-64|            2|                          1|           55-64|                             2|         17820|        CO|      08041|        565000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|          31|     355000|        2.875|      180|           1|   565|        55-64|            1|                          2|           55-64|                             1|         11244|        CA|      06059|        855000|          1|             3|
|         2020|549300RBX56T2MW5HO19|        1|          31|     265000|          3.0|      360|           1|    57|        65-74|            1|                          5|           65-74|                             1|         19740|        CO|      08031|        355000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|          31|     485000|        2.249|      180|           1|   171|        55-64|            1|                          1|           55-64|                             1|         41740|        CA|      06073|        995000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|          31|     345000|        2.875|      360|           1|   157|        25-34|            1|                          5|           25-34|                             5|         24540|        CO|      08123|        365000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|          32|     475000|          3.0|      360|           1|   285|        35-44|            1|                          1|           35-44|                             1|         16984|        IL|      17043|        635000|          1|             1|
|         2020|549300RBX56T2MW5HO19|        1|          31|     365000|        2.875|      360|     

In [15]:
df = df.drop('activity_year')

### 3. Data Checks

#### Data Type

In [16]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, FloatType

from pyspark.sql.functions import col, when, count

In [17]:

coltype_map = {
    "lei": StringType(),
    "loan_type": IntegerType(),
    "loan_purpose": IntegerType(),
    "loan_amount": FloatType(),
    "interest_rate": FloatType(),
    "loan_term": IntegerType(),
    "action_taken": IntegerType(),
    "income": FloatType(),
    "applicant_age": StringType(),
    "applicant_sex": IntegerType(),
    "applicant_credit_score_type": IntegerType(),
    "co_applicant_age": StringType(),
    "co_applicant_credit_score_type": IntegerType(),
    "derived_msa_md": IntegerType(),
    "state_code": StringType(),
    "county_code": StringType(),
    "property_value": FloatType(),
    "total_units": IntegerType(),
    "occupancy_type": IntegerType(),
}



In [18]:
for col_name, data_type in coltype_map.items():
    df = df.withColumn(col_name, col(col_name).cast(data_type))

In [19]:
df.printSchema()

root
 |-- lei: string (nullable = true)
 |-- loan_type: integer (nullable = true)
 |-- loan_purpose: integer (nullable = true)
 |-- loan_amount: float (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- loan_term: integer (nullable = true)
 |-- action_taken: integer (nullable = true)
 |-- income: float (nullable = true)
 |-- applicant_age: string (nullable = true)
 |-- applicant_sex: integer (nullable = true)
 |-- applicant_credit_score_type: integer (nullable = true)
 |-- co_applicant_age: string (nullable = true)
 |-- co_applicant_credit_score_type: integer (nullable = true)
 |-- derived_msa_md: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- county_code: string (nullable = true)
 |-- property_value: float (nullable = true)
 |-- total_units: integer (nullable = true)
 |-- occupancy_type: integer (nullable = true)



#### Missing values

In [20]:
from pyspark.sql.functions import col, isnan

# Missing count for each column
missing_info = df.select([
    count(when(col(c).isNull() | isnan(col(c)) |(col(c) == "NA"), c)).alias(c) for c in df.columns
])


In [21]:
missing_info

+---+---------+------------+-----------+-------------+---------+------------+-------+-------------+-------------+---------------------------+----------------+------------------------------+--------------+----------+-----------+--------------+-----------+--------------+
|lei|loan_type|loan_purpose|loan_amount|interest_rate|loan_term|action_taken| income|applicant_age|applicant_sex|applicant_credit_score_type|co_applicant_age|co_applicant_credit_score_type|derived_msa_md|state_code|county_code|property_value|total_units|occupancy_type|
+---+---------+------------+-----------+-------------+---------+------------+-------+-------------+-------------+---------------------------+----------------+------------------------------+--------------+----------+-----------+--------------+-----------+--------------+
|  0|        0|           0|          0|      8058681|   787724|           0|3805537|            0|            0|                          0|               0|                             0|             0|    191662|     301904|       6256159|      67537|             0|
+---+---------+------------+-----------+-------------+---------+------------+-------+-------------+-------------+---------------------------+----------------+------------------------------+--------------+----------+-----------+--------------+-----------+--------------+

In [22]:
# Median imputation for 'income'
from pyspark.ml.feature import Imputer

income_imputer = Imputer(inputCols=['income'], outputCols=['income'], strategy="median")
df = income_imputer.fit(df).transform(df)

In [23]:
# Mean imputation for `interest_rate`
interest_rate_imputer = Imputer(inputCols=["interest_rate"], outputCols=["interest_rate"], strategy="mean")

# Fit and transform
df = interest_rate_imputer.fit(df).transform(df)

In [24]:
# Mode imputation for 'loan_Term'
from pyspark.sql.functions import col, count, when

# Calculate mode for `loan_term`
mode_loan_term = (
    df.groupBy("loan_term")
    .count()
    .orderBy(col("count").desc())
    .first()[0]
)

# Replace missing values with the mode
df = df.withColumn( "loan_term", when(col("loan_term").isNull(), mode_loan_term).otherwise(col("loan_term")))


In [25]:
# Remove rows that has no 'property_value'
df = df.filter(col('property_value')!=0)

In [26]:
# Remove rows that has no 'state_code'
df = df.filter(col('state_code')!='NA')

In [27]:
# Remove rows that has no 'county_code'
df = df.filter(col('county_code')!='NA')

In [28]:
# Remove rows that has no 'total_units'
df = df.filter(col('total_units')!=0)

In [29]:
df.count()

19216487

In [30]:
df

+--------------------+---------+------------+-----------+-------------+---------+------------+------+-------------+-------------+---------------------------+----------------+------------------------------+--------------+----------+-----------+--------------+-----------+--------------+
|                 lei|loan_type|loan_purpose|loan_amount|interest_rate|loan_term|action_taken|income|applicant_age|applicant_sex|applicant_credit_score_type|co_applicant_age|co_applicant_credit_score_type|derived_msa_md|state_code|county_code|property_value|total_units|occupancy_type|
+--------------------+---------+------------+-----------+-------------+---------+------------+------+-------------+-------------+---------------------------+----------------+------------------------------+--------------+----------+-----------+--------------+-----------+--------------+
|549300RBX56T2MW5HO19|        1|           1|   135000.0|        3.875|      360|           1|  65.0|        25-34|            1|                          2|           25-34|                             5|         29620|        MI|      26065|      145000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|          31|   315000.0|        2.124|      180|           1| 135.0|        45-54|            1|                          5|           45-54|                             1|         31084|        CA|      06037|      855000.0|          1|             1|
|549300RBX56T2MW5HO19|        2|           1|   135000.0|         3.43|      360|           1|  95.0|        65-74|            1|                          9|           65-74|                             9|         41100|        UT|      49053|      255000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|           1|   435000.0|          2.5|      360|           1| 328.0|        55-64|            2|                          1|           55-64|                             2|         17820|        CO|      08041|      565000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|          31|   355000.0|        2.875|      180|           1| 565.0|        55-64|            1|                          2|           55-64|                             1|         11244|        CA|      06059|      855000.0|          1|             3|
|549300RBX56T2MW5HO19|        1|          31|   265000.0|          3.0|      360|           1|  57.0|        65-74|            1|                          5|           65-74|                             1|         19740|        CO|      08031|      355000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|          31|   485000.0|        2.249|      180|           1| 171.0|        55-64|            1|                          1|           55-64|                             1|         41740|        CA|      06073|      995000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|          31|   345000.0|        2.875|      360|           1| 157.0|        25-34|            1|                          5|           25-34|                             5|         24540|        CO|      08123|      365000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|          32|   475000.0|          3.0|      360|           1| 285.0|        35-44|            1|                          1|           35-44|                             1|         16984|        IL|      17043|      635000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|          31|   365000.0|        2.875|      360|           1| 136.0|        25-34|            2|                          5|           25-34|                             1|         38900|        OR|      41051|      465000.0|          1|             1|
|549300RBX56T2MW5HO19|        1|          31|   275000.0|        3.125|      360|           1|  91.0|        65-74|            1|                          2|           55-64|                             2|         19740|        CO|      08031|      305000.0|          1|            

Target variable

In [31]:
df = df.withColumn('action_taken', when(col('action_taken')!=1, 0).otherwise(1) )

In [32]:
df.count()

19216487

In [33]:
# count number of loan approved
df.filter(col('action_taken')==1).count()

13569956

In [34]:
# count of loan rejected
df.filter(col('action_taken')==0).count()

5646531

#### Check unique values of each column

In [25]:
df.select('action_taken').distinct()

+------------+
|action_taken|
+------------+
|           1|
|           0|
+------------+

In [27]:
df.select('lei').distinct().count()

2957

In [28]:
df.select('applicant_age').distinct().count()

9

In [29]:
df.select('co_applicant_age').distinct().count()

9

In [30]:
df.select('state_code').distinct().count()

54

In [31]:
df.select('county_code').distinct().count()

3222

In [32]:
df.select('loan_type').distinct().count()

4

In [33]:
df.select('loan_purpose').distinct().count()

6

In [34]:
df.select('applicant_sex').distinct().count()

5

In [35]:
df.select('applicant_credit_score_type').distinct().count()

10

In [36]:
df.select('derived_msa_md').distinct().count()

413

In [37]:
df.select('total_units').distinct().count()

4

In [38]:
df.select('occupancy_type').distinct().count()

3

In [34]:
# from pyspark.sql.functions import col, isnan

# df.filter(col('action_taken').isNull()| isnan(col('action_taken')))

### Load data to parquet

In [35]:
df.write.parquet('./cleaned_dataset', mode='overwrite')

24/12/03 16:13:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:38 WARN MemoryManager: Total allocation exceeds 95.00% 